<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Pytorch_WithDeeplizard/blob/master/26_CNN_Training_With_Code_Example_Neural_Network_Programming_Course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN Training Process
在这节中，我们将学习训练卷积神经网络所需的步骤。

到目前为止，我们了解了Tensors，也了解了有关PyTorch神经网络的所有知识。 现在，我们准备开始训练过程。

* 准备数据

* 建立模型

* 训练模型

  * 计算损耗，梯度并更新权重

* 分析模型的结果


## 1.训练：正向传播后做什么
在训练过程中，我们会进行正向传播，那之后又会怎样呢？ 我们假设我们得到了一个批处理并将其通过网络正向传播。 一旦获得输出，我们就将预测输出与实际标签进行比较，并且一旦我们知道预测标签与实际标签的距离有多近，就可以调整网络内部的权重，以使网络预测的值更加接近到真实值（标签）。

所有这些都是针对单个批次的，我们将对每个批次重复此过程，直到我们涵盖了训练集中的每个样本为止。 

* 在完成所有批次的此过程并传递训练集中的每个样本后，我们说一个纪元已经完成。 我们使用**“ epoch”**一词来表示一个涵盖**整个训练集**的时间段。

在整个培训过程中，我们会根据需要执行尽可能多的时间，以达到我们期望的准确性水平。 这样，我们可以执行以下步骤：

* 1.从训练集中获取批次。

* 2.将批处理传递到网络。

* 3.计算损失（预测值和真实值之间的差）。

* 4.用网络权重计算损失函数的梯度。

* 5.使用梯度更新权重以减少损失。

* 6.重复步骤1-5，直到完成一个纪元。

* 7.重复步骤1-6，以达到达到最小损耗所需的次数。

我们已经完全知道如何执行步骤1和2。如果您已经学习了深度学习基础知识系列，那么您知道我们使用损失函数来执行步骤3，并且您知道我们使用反向传播和优化算法来执行步骤4和5。步骤6和7只是标准的Python循环（训练循环）。 让我们看看如何在代码中完成此操作。


## 2.训练过程

因为我们在前几节中中禁用了PyTorch的梯度跟踪功能，所以我们需要确保重新启用它（默认情况下是启用的）。

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [ ]:
torch.set_grad_enabled(True)

### 准备正向传播
---
我们已经知道如何获得一批并通过网络将其进行前向传播。让我们看看正向传播完成后我们该怎么做。

我们将从以下内容开始：

* 创建网络类的实例。

* 创建一个数据加载器，从我们的训练集中提供大小为100的批。

* 打开其中一批的图像和标签。

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        # (1) input layer
        t = t
        print("input layer shape:",t.shape)

        # (2) hidden conv layer
        t = self.conv1(t)
        print("conv1 layer shape:",t.shape)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        print("max_pool layer shape:",t.shape)
        # (3) hidden conv layer
        t = self.conv2(t)
        print("conv2 layer shape:",t.shape)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        print("flatten layer shape:",t.shape)
        t = self.fc1(t)
        print("linear1 layer shape:",t.shape)
        t = F.relu(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        print("linear2 layer shape:",t.shape)
        t = F.relu(t)

        # (6) output layer
        t = self.out(t)
        print("out layer shape:",t.shape)
        #t = F.softmax(t, dim=1)

        return t

In [ ]:
train_set = torchvision.datasets.FashionMNIST(root='./data',train=True,
                                download=True,
                                transform=transforms.Compose([
        transforms.ToTensor()
    ]))


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw




Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
network=Network()

In [ ]:
train_loader =torch.utils.data.DataLoader(train_set,batch_size=100)
batch = next(iter(train_loader))
images,labels = batch
images.shape

torch.Size([100, 1, 28, 28])

接下来，我们准备通过网络将我们的一批图像正向传播，并获得输出预测。一旦我们有了预测张量，我们就可以使用预测和真实标签来计算损失。

### 计算损失
---
为此，我们将使用PyTorch的nn.functional API中可用的cross_entropy（）损失函数。 一旦我们有了损失，我们就可以打印它，并使用我们之前创建的函数检查正确的预测数。

In [ ]:
preds = network(images)
loss = F.cross_entropy(preds,labels)

input layer shape: torch.Size([100, 1, 28, 28])
conv1 layer shape: torch.Size([100, 6, 24, 24])
max_pool layer shape: torch.Size([100, 6, 12, 12])
conv2 layer shape: torch.Size([100, 12, 8, 8])
flatten layer shape: torch.Size([100, 192])
linear1 layer shape: torch.Size([100, 120])
linear2 layer shape: torch.Size([100, 60])
out layer shape: torch.Size([100, 10])


In [ ]:
loss.item()

2.301669120788574

In [ ]:
def get_num_correct(preds,labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [ ]:
get_num_correct(preds,labels)

3

cross_entropy（）函数返回了一个标量值，因此我们使用item（）方法将loss打印为Python数字。我们在100个预测类中有3个是正确的，因为我们有10个预测类，这是我们随机猜测的结果。

### 计算梯度
---
使用PyTorch可以很容易地计算出梯度。 由于我们的网络是PyTorch nn.Module，因此PyTorch在后台创建了一个计算图。 随着我们的张量流过我们的网络，所有计算都添加到图中。 然后，PyTorch使用计算图来计算损失函数相对于网络权重的梯度。

在计算梯度之前，让我们验证一下conv1层内部当前没有梯度。 gradients是可在每一层的权重张量的grad（gradients的缩写）属性中访问的张量。


In [ ]:
print(network.conv1.weight.grad)

None


要计算梯度，我们在loss张量上调用backward（）方法，如下所示：

In [ ]:
loss.backward()

现在，损失函数的梯度已存储在权重张量内部。

In [ ]:
network.conv1.weight.grad.shape

torch.Size([6, 1, 5, 5])

**优化器使用这些梯度来更新各个权重**。 要创建优化器，我们使用torch.optim包，该包具有许多可以使用的优化算法实现。 我们将以Adam为例。

### 更新权重
---
对于Adam类构造函数，我们传递网络参数（这是优化器访问梯度的方式），并且传递学习率。

最后，我们要做的就是更新权重，就是告诉优化器使用梯度下降来逐步实现损失函数的最小值。

In [ ]:
import torch.optim as optim

In [ ]:
optimizer =optim.Adam(network.parameters(),lr = 0.01)
optimizer.step()# 更新权重

调用step（）函数时，优化器将使用存储在网络参数中的梯度来更新权重。 这意味着如果我们再次通过网络传递同一批次，我们应该期望减少损失。 检查一下，我们可以看到确实是这样：

In [ ]:
preds = network(images)

input layer shape: torch.Size([100, 1, 28, 28])
conv1 layer shape: torch.Size([100, 6, 24, 24])
max_pool layer shape: torch.Size([100, 6, 12, 12])
conv2 layer shape: torch.Size([100, 12, 8, 8])
flatten layer shape: torch.Size([100, 192])
linear1 layer shape: torch.Size([100, 120])
linear2 layer shape: torch.Size([100, 60])
out layer shape: torch.Size([100, 10])


In [ ]:
loss.item()

2.301669120788574

In [ ]:
loss = F.cross_entropy(preds,labels)
loss.item()

2.2768006324768066

In [ ]:
get_num_correct(preds,labels)

18


## 3.训练一批数据

我们可以通过以下方式总结用于单批训练的代码：

In [ ]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set,batch_size=100)

optimizer = optim.Adam(network.parameters(),lr=0.01)

batch = next(iter(train_loader))
images,labels = batch

preds = network(images)
loss = F.cross_entropy(preds,labels)

loss.backward()
optimizer.step()

print('loss1:', loss.item())
preds = network(images)
loss = F.cross_entropy(preds, labels)
print('loss2:', loss.item())

input layer shape: torch.Size([100, 1, 28, 28])
conv1 layer shape: torch.Size([100, 6, 24, 24])
max_pool layer shape: torch.Size([100, 6, 12, 12])
conv2 layer shape: torch.Size([100, 12, 8, 8])
flatten layer shape: torch.Size([100, 192])
linear1 layer shape: torch.Size([100, 120])
linear2 layer shape: torch.Size([100, 60])
out layer shape: torch.Size([100, 10])
loss1: 2.309680700302124
input layer shape: torch.Size([100, 1, 28, 28])
conv1 layer shape: torch.Size([100, 6, 24, 24])
max_pool layer shape: torch.Size([100, 6, 12, 12])
conv2 layer shape: torch.Size([100, 12, 8, 8])
flatten layer shape: torch.Size([100, 192])
linear1 layer shape: torch.Size([100, 120])
linear2 layer shape: torch.Size([100, 60])
out layer shape: torch.Size([100, 10])
loss2: 2.2806639671325684


## 下一步是建立训练循环

我们现在应该对训练过程有一个很好的了解。在下一集中，我们将看到如何通过构建训练循环来完成过程来扩展这些想法。下次见